# Prompt Declaration Language

Prompt engineering is difficult: minor variations in prompts have large impacts on the output of LLMs and prompts are model-dependent. In recent years <i> prompt programming languages </i> have emerged to bring discipline to prompt engineering. Many of them are embedded in an imperative language such as Python or TypeScript, making it difficult for users to directly interact with prompts and multi-turn LLM interactions.

The Prompt Declaration Language (PDL) is a YAML-based declarative approach to prompt programming, where prompts are at the forefront. PDL facilitates model chaining and tool use, abstracting away the plumbing necessary for such compositions, enables type checking of the input and output of models, and is based on LiteLLM to support a variety of model providers. PDL has been used with RAG, CoT, ReAct, and an agent for solving SWE-bench. PDL is [open-source](https://github.com/IBM/prompt-declaration-language) and works well with watsonx.ai and Granite models.

You can use PDL stand-alone or from a Python SDK or, as shown here, in a notebook via a notebook extension. In the cell output, model-generated text is rendered in green font, and tool-generated text is rendered in purple font.

In [ ]:
! pip install prompt-declaration-language
! pip install 'prompt-declaration-language[examples]'

In [2]:
%load_ext pdl.pdl_notebook_ext

## Model call

In PDL, the user specifies step-by-step the shape of data they want to generate. In the following, the `text` construct indicates a text block containing a prompt and a model call. Implicitly, PDL builds a background conversational context (list of role/content) which is used to make model calls. Each model call uses the context built so far as its input prompt.

In [15]:
%%pdl --reset-context
description: Model call
text: 
- "What is the meaning of life?\n"
- model: replicate/ibm-granite/granite-8b-code-instruct-128k
  parameters:
    stop_sequences: "!"
    include_stop_sequence: true

What is the meaning of life?
The meaning of life is a philosophical question that has been debated by many thinkers throughout history. There is no one definitive answer, as the answer may vary depending on one's personal beliefs, values, and experiences.


## Model chaining
Model chaining can be done by simply adding to the list of models to call declaratively. Since this cell has the `%%pdl` cell magic without `--reset-context`, it executes in the context created by the previous cell.

In [17]:
%%pdl
text:
- "\nSay it like a poem\n"
- model: replicate/ibm-granite/granite-8b-code-instruct-128k
- "\n\nWhat is the most important verse in this poem?\n"
- model: replicate/ibm-granite/granite-8b-code-instruct-128k


Say it like a poem
The meaning of life, a question so profound,
A mystery that has puzzled men and women for so long,
A path that we must tread, a goal to reach,
A journey that will bring us joy and pain,

A road that twists and turns, a fork in the road,
Where we must choose, which way to go,
A decision that we must make, with our souls at stake,
A choice that will shape our destiny,

The meaning of life, a question so grand,
A goal that we must strive for, to find,
A purpose that gives our hearts meaning,
A reason to live, a reason to die,

A journey that will take us far, a journey that will bring,
A new understanding of the world we live in,
A new perspective on life, a new way of thinking,
A new path to follow, a new way to live,

The meaning of life, a question so deep,
A mystery that will never be solved,
A journey that will take us far, a journey that will bring,
A new understanding of the world we live in,

A road that twists and turns, a fork in the road,
Where we must choos

## Chat templates

The following example shows a full-fledged chatbot. In PDL roles are high level annotations and PDL takes care of applying the appropriate chat templates. This example illustrates the use of control structures such as the repeat-until block and reading from files or stdin with the read block. The chatbot repeatedly prompts the user for a query, which it submits to a model, and stops when the query is quit.

In [18]:
%%pdl
text:
- role: system
  content: You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.
- "Type `quit` to exit this chatbot.\n"
- repeat:
    text:
    - read:
      message: ">>> "
      def: query
      contribute: [context]
    - model: replicate/ibm-granite/granite-8b-code-instruct-128k
  until: ${ query == 'quit'}
  join:
    with: "\n\n"
role: user


You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.Type `quit` to exit this chatbot.


>>>  What is APR?


APR stands for Annual Percentage Rate. It is a measure of the total cost of borrowing money, including interest and fees, expressed as a yearly rate. It is commonly used in the lending industry to compare the cost of different loans and credit products.




>>>  Say it like I'm 5 years old


The meaning of life is like a big, big, big question mark. It's a question that has been asked for as long as people can remember, and it's still a question that people don't always know the answer to. Some people think the answer is to have fun and make friends, while others think the answer is to work hard and be smart. But no matter what the answer is, the question of what gives life meaning is a question that will always be with us.




>>>  quit


Thank you for chatting with me! If you have any more questions or need further assistance, feel free to ask.


## Chat templates

The first call to the model in the above program submits the following prompt. PDL takes care of applying the appropriate chat templates and tags, and builds the background context implicitly. Chat templates make your program easier to port across models, since you do not need to specify control tokens by hand. All the user has to do is list the models they want to chain, PDL takes care of the rest.

```
<|start_of_role|>system<|end_of_role|>You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>Type `quit` to exit this chatbot.
What is APR?<|end_of_text|><|start_of_role|>assistant<|end_of_role|>
```

## Data pipeline

The following program shows a common prompting pattern: read some data, formulate a prompt using that data, submit to a model, and evaluate. In this program, we formulate a prompt for code explanation. The program first defines two variables: `code`, which holds the data we read, and `truth` for the ground truth. It then prints out the source code, formulates a prompts with the data, and calls a model to get an explanation. Finally, a Python code block uses the Levenshtein text distance metric and evaluate the explanation against the ground truth. This pipeline can similarly be applied to an entire data set to produce a jsonl file.

In [19]:
%%pdl --reset-context
description: Code explanation example
defs:
  CODE:
    read: ./data.yaml
    parser: yaml
  TRUTH:
    read: ./ground_truth.txt
text:
- "\n${ CODE.source_code }\n"
- model: replicate/ibm-granite/granite-8b-code-instruct-128k
  def: EXPLANATION
  input: |
      Here is some info about the location of the function in the repo.
      repo: 
      ${ CODE.repo_info.repo }
      path: ${ CODE.repo_info.path }
      Function_name: ${ CODE.repo_info.function_name }


      Explain the following code:
      ```
      ${ CODE.source_code }```
- |


  EVALUATION:
  The similarity (Levenshtein) between this answer and the ground truth is:
- def: EVAL
  lang: python
  code: |
    import textdistance
    expl = """
    ${ EXPLANATION }
    """
    truth = """
    ${ TRUTH }
    """
    result = textdistance.levenshtein.normalized_similarity(expl, truth)


@SuppressWarnings("unchecked")
public static Map<String, String> deserializeOffsetMap(String lastSourceOffset) throws IOException {
  Map<String, String> offsetMap;
  if (lastSourceOffset == null || lastSourceOffset.isEmpty()) {    
    offsetMap = new HashMap<>();  
  } else {
    offsetMap = JSON_MAPPER.readValue(lastSourceOffset, Map.class);  
  }
  return offsetMap;
}

The function `deserializeOffsetMap` is a method that takes a string `lastSourceOffset` as input and returns a `Map` of `String` keys and `String` values. The function is used to deserialize a JSON string into a `Map` object.
 

 The function first checks if the `lastSourceOffset` is null or empty. If it is, it creates a new `HashMap` object and assigns it to the `offsetMap` variable. If the `lastSourceOffset` is not null or empty, the function uses the `JSON_MAPPER` object to read the JSON string and deserialize it into a `Map` object. The `JSON_MAPPER` object is assumed to be a pre-defined object that is used for J

## Conclusion

Since prompts are at the forefront, PDL makes users more productive in their trial-and-error with LLMs. Try it!

https://github.com/IBM/prompt-declaration-language